# Importing necessary libraries to run models

In [1]:
import numpy as np
from pickle import dump
from keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Embedding

Using TensorFlow backend.


# Need to load my 5 txt files to make the 5 models

In [2]:
def loader(textfile):
    # Open the file as read-only
    file = open(textfile, 'r')
    # Actually read the file
    text = file.read()
    # Now that I have the data in my text variable, need to close out of the file
    file.close()
    return text

In [4]:
# Create the 5 instances of the 5 ratings

one = loader('One_Star.txt')
# two = loader('Two_Star.txt')
# three = loader('Three_Star.txt')
# four = loader('Four_Star.txt')
# five = loader('Five_Star.txt')

In [5]:
# Need to split up each of these data sets into training sequences by the different lines. 
# End goal is to have user input 5 words and the text generator will work its magic

In [6]:
one_lines = one.split('\n')
# two_lines = two.split('\n')
# three_lines = three.split('\n')
# four_lines = four.split('\n')
# five_lines = five.split('\n')

In [7]:
# Each line has been broken out with 5 sequence of words
one_lines[:5]

['atrocious the live music was loud',
 'the live music was loud and',
 'live music was loud and mediocre',
 'music was loud and mediocre the',
 'was loud and mediocre the decor']

# Need to convert the words in my data to integers in order to be used by the model. 

Will need to train the keras tokenizer on entire dataset to assign a unique ID to each unique word in the dataset

In [8]:
tokenizer = Tokenizer()

def word_to_int(lines):
    tokenizer.fit_on_texts(lines)
    sequences = tokenizer.texts_to_sequences(one_lines)
    # Need the total size of the vocabulary for our embedding layer in model
    vocab_size = len(tokenizer.word_index) + 1
    return sequences, vocab_size

In [18]:
# Running the function for all 5 data sets
one_sequence, one_vocab_size = word_to_int(one_lines)
# two_sequence, two_vocab_size = word_to_int(two_lines)
# three_sequence, three_vocab_size = word_to_int(three_lines)
# four_sequence, four_vocab_size = word_to_int(four_lines)
# five_sequence, five_vocab_size = word_to_int(five_lines)

In [19]:
# each line has been converted to unique ints to represent each word
one_sequence[:5]

[[2408, 1, 547, 515, 7, 773],
 [1, 547, 515, 7, 773, 3],
 [547, 515, 7, 773, 3, 1012],
 [515, 7, 773, 3, 1012, 1],
 [7, 773, 3, 1012, 1, 1130]]

# Need to convert my sequences into inputs and outputs to structure my model to predict words based on previous set of words

## One Star Review Modeling

In [20]:
one_sequence = np.array(one_sequence)
X, y = one_sequence[:,:-1], one_sequence[:,-1]
y = to_categorical(y, num_classes=one_vocab_size)
seq_length = X.shape[1]

In [21]:
# defining my model

model = Sequential()
model.add(Embedding(one_vocab_size, 5, input_length=seq_length))
model.add(LSTM(100, return_sequences=True))
model.add(LSTM(100))
model.add(Dense(100, activation='relu'))
model.add(Dense(one_vocab_size, activation = 'softmax'))
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 5, 5)              129250    
_________________________________________________________________
lstm_3 (LSTM)                (None, 5, 100)            42400     
_________________________________________________________________
lstm_4 (LSTM)                (None, 100)               80400     
_________________________________________________________________
dense_3 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_4 (Dense)              (None, 25850)             2610850   
Total params: 2,873,000
Trainable params: 2,873,000
Non-trainable params: 0
_________________________________________________________________
None


In [22]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X,y,batch_size=128, epochs = 100)

Epoch 1/100
 15360/200000 [=>............................] - ETA: 3:12 - loss: 7.8956 - acc: 0.0417

KeyboardInterrupt: 